In [1]:
from functions4 import *
import matplotlib.colors as colors
from scipy.integrate import dblquad
from scipy import interpolate

In [59]:
GAMMA,Phi=get_best_fit_values()
mc_all=[]
DETECTOR=LoadDETECTOR("1111")
localPath="/net/scratch_icecube4/user/glauch/"
datapath     = localPath+"DATA/"
SAVE_DIRECTORY = "/net/scratch_icecube4/user/kalaczynski/Plots/"
#mc1=simulationLoad(DETECTOR[0], datapath, 2.13)[0] # getting MC for each dataset
mc2=simulationLoad(DETECTOR[1], datapath, 2.13)[0]
#mc3=simulationLoad(DETECTOR[2], datapath, 2.13)[0]
#mc4=simulationLoad(DETECTOR[3], datapath, 2.13)[0]
#mc_all.append(mc1)
#mc_all.append(mc2)
#mc_all.append(mc3)
#mc_all.append(mc4)

/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_11 the total uptime is 29556432.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2011_10602_11077_11191_corrected_ow.npy
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC79 the total uptime is 26784000.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC79_6308_6850_corrected_ow.npy


In [64]:
bins=100
EBinWidth=10**9-10**2
SinDecEdges, deltaSinDec = np.linspace(np.sin(np.radians(-6)),1,bins, retstep=True)
deltaSinDec = (SinDecEdges[-1]-SinDecEdges[0])/(len(SinDecEdges)-1)
solidAngle = deltaSinDec*2.*np.pi
w_aeff = mc2["ow"]/EBinWidth/solidAngle

H, SinDecEdges = np.histogram(np.sin(mc2["trueDec"]), bins=SinDecEdges, weights=w_aeff)
H = np.ma.masked_array(H)
H.mask = H == 0

x=np.linspace(np.sin(np.radians(-6)),1,num=len(H))

# interpolation  with a spline:
Aeff_theta_interpolated = interpolate.splrep(x, H) 
tck = interpolate.splrep(x, H)
x_new=np.linspace(np.sin(np.radians(-6)),1,num=1000*len(H))
y_new = interpolate.splev(x_new, tck, der=0)

In [60]:
print np.sqrt(len(mc2["trueDec"]))

1176.54579171


In [70]:
# Effective area versus the declination angle plot:

#plt.scatter(x,H)
plt.scatter(np.degrees(np.arcsin(x_new)),y_new)
#plt.xlim([-6, 90])
#plt.ylim(0, 10**8)
#plt.yscale('log', basey = 10)
plt.xlabel("$\\theta \,[^\circ]$", fontsize=34)
plt.ylabel(r"$A_{\mathsf{eff}} (\theta)\,[\mathsf{cm}^2]$", fontsize=34)
#plt.savefig(SAVE_DIRECTORY+"A_eff_scatter_log.png", dpi=300)
plt.show()

# ready code for getEffAreaVsDec() used in functions.py (OLD VERSION):

In [9]:

datapath="/net/scratch_icecube4/user/glauch/DATA/"
DETECTOR=LoadDETECTOR("1111")
index=2.13 # assumed spectral index

Aeff_res=plot_aeff_vs_dec_energy(simulationLoad(DETECTOR[1], datapath, 2.13)[0], logE_range=(2,9), sinDec_range=(np.sin(np.radians(-6)),1), bins=[40,40], title=None, savepath=None)

X,Y=np.meshgrid(Aeff_res[2],Aeff_res[1])
H=Aeff_res[0]

Aeff= np.sum(H, axis=1)*(X[0][1]-X[0][0])
deltaE=np.array([10**Aeff_res[1][i+1]-10**Aeff_res[1][i] for i in range(len(Aeff_res[2])-1)])
E=np.array([(10**Aeff_res[1][i+1]+10**Aeff_res[1][i])/2 for i in range(len(Aeff_res[2])-1)])
Aeff_theta=[np.sum((E**(-index))*np.concatenate(H[:,i:i+1])*deltaE)/np.sum((E**(-index))*deltaE) for i in range(len(X)-1)]

x=np.degrees(np.arcsin(np.linspace(np.sin(np.radians(-6)),1.,num=len(Aeff_theta))))
Aeff_theta_interpolated = interp1d(x, Aeff_theta)

/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_11 the total uptime is 29556432.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2011_10602_11077_11191_corrected_ow.npy
EBinWidth 1384260 MC  length 1384260


# Rene's code:

In [38]:
from scipy.interpolate import UnivariateSpline
mc=mc2
logE_range=[(2,9)]
sinDec_range=(-1,1)
bins=20
title=None
savepath=None

sinDecEdges = np.linspace(sinDec_range[0],sinDec_range[1],bins, endpoint=True)
deltaSinDec = (sinDecEdges[-1]-sinDecEdges[0])/(len(sinDecEdges)-1)
solidAngle = deltaSinDec*2*np.pi

aeff = {}
  
for eRange in logE_range:
    eRange = np.array(eRange)
    mask1 = set(np.where(np.log10(mc["trueE"]) > eRange.min())[0])
    mask2 = set(np.where(np.log10(mc["trueE"]) < eRange.max())[0])
    mask = list(mask1 & mask2)   
    EBinWidth = pow(10, eRange[-1]) - pow(10, eRange[0]) 

H, sinDecEdges = np.histogram(np.sin(mc["trueDec"][mask]), bins=sinDecEdges, weights=1e-4*mc["ow"][mask]/EBinWidth/solidAngle)
spline = UnivariateSpline((sinDecEdges[1:]+sinDecEdges[:-1])/2., np.maximum(np.log10(H), -7), s=1e-3)
aeff[tuple(eRange)] = (H, sinDecEdges, spline)
   
plt.figure(figsize=(16,9))
sinDec = np.linspace(-1,1.,100)
for eRange in logE_range:
    H, edges, spline = aeff[tuple(eRange)]
    hist_line, = plt.plot(edges, np.concatenate([H,[0]]), drawstyle="steps-post", label=r"${}$".format(eRange[0])+r"$\leq\log(E/\mathrm{GeV})\leq$"+"${}$".format(eRange[1]), ls="-")
    plt.plot(sinDec, 10000*10**spline(sinDec), ls="--", color=hist_line.get_color())
   
plt.xlabel(r"$\sin(\delta)$")
#plt.yscale("log", nonposy="clip")
plt.xlim([np.sin(np.radians(-6)), 1])
#plt.ylim([1e-4, 1e4])
#plt.ylabel(r"$\log_{10}(A_{eff}/ \mathrm{m}^2)$")
plt.ylabel(r"$A_{eff}/ \mathrm{cm}^2$")
#plt.legend(loc=2)
#plt.savefig(SAVE_DIRECTORY+"A_eff_scatter_rene.png", dpi=300)
plt.show() 


# Junk

In [106]:
bins=[20,20]

# arbitrary bins
EEdges, deltaE = np.linspace(10**2,10**9, bins[0], retstep=True)
DecEdges, deltaDec = np.linspace(np.radians(-6),np.pi/2,bins[1], retstep=True)
center_E_bins = np.array(EEdges[:-1])+deltaE/2
center_Dec_bins = np.array(DecEdges[:-1]) + deltaDec/2

# to get weights:
lowerEEdge = np.floor((mc2["trueE"] - EEdges[0])/deltaE)*deltaE + EEdges[0]
upperEEdge = lowerEEdge + deltaE
EBinWidth = upperEEdge - lowerEEdge
deltaSinDec = (np.sin(DecEdges[-1])-np.sin(DecEdges[0]))/(len(DecEdges)-1)
solidAngle = deltaSinDec*2.*np.pi
w_aeff = mc2["ow"]/EBinWidth/solidAngle
#*(mc2["trueE"])**(-0.13) cuz ow are for GAMMA=2.0
#H, EEdges, DecEdges = np.histogram2d(mc2["trueE"], mc2["trueDec"], bins=[EEdges, DecEdges], weights=w_aeff)
#H = np.ma.masked_array(H)
#H.mask = H == 0

H, DecEdges = np.histogram(np.sin(mc2["trueDec"]), bins=DecEdges, weights=mc2["ow"])
H = np.ma.masked_array(H)
H.mask = H == 0